In [1]:
import scipy.io
data = scipy.io.loadmat('D:/本科研究生资料/硕士研究生/课程/机器学习与数据科学/作业/YaleB_32x32.mat')

In [2]:
print(data['index'][0])

[ 1  1  1 ... 38 38 38]


In [3]:
def parse(data):
    data_set = data['X']
    label = data['index'][0]
    data_num = len(label)
    dimensions = len(data['X'])
    classes = len(set(label))
    return data_set,label,data_num,dimensions,classes

data_set,label,data_num,dimensions,classes = parse(data)
print(classes)

38


In [4]:
from random import sample
def split_train_test(data_num):
    l = range(data_num)
    train_set = sorted(sample(l,int(data_num/2)))
    test_set = sorted(set(l) - set(train_set))
    return train_set,test_set

train_set,test_set = split_train_test(data_num)
# print(train_set)
# print(test_set)

In [5]:
def Odistance(x,y):
    distance = 0
    dimensions = len(x)
    for dim in range(dimensions):
        distance += (x[dim] - y[dim])**2
    return distance**0.5

#print(Odistance([1,1,1,1],[2,2,2,2]))

In [6]:
from cvxopt import matrix, solvers

def judge(k_list):
    k_index_list = [x[0] for x in k_list]
    class_ = [0]*(classes+1)
    for index in k_index_list:
        class_[label[index]] += 1
    return class_.index(max(class_))

def judge2(k_list):
    class_ = [0]*(classes+1)
    d1 = k_list[0][1]
    dk = k_list[-1][1]
    for index_distance in k_list:
        index = index_distance[0]
        di = index_distance[1]
        if dk - d1 != 0:
            wi = (dk - di)/(dk - d1)
        else:
            wi = 1
        class_[label[index]] += wi
    return class_.index(max(class_))

def judge3(k_list,x):
    k_index_list = [x[0] for x in k_list]
    k = len(k_index_list)
    class_ = [0]*(classes+1)
    P = []
    q = []
    G = []
    h = []
    A = []
    b = []
    for index1 in k_index_list:
        p_row = []
        for index2 in k_index_list:
            p_value = 0.0
            for dim in range(dimensions):
                p_value += 2*data_set[dim][index1]*data_set[dim][index2]
            p_row.append(p_value)
        P.append(p_row)
        
    for index in k_index_list:
        q_value = 0.0
        for dim in range(dimensions):
            q_value += -2*x[dim]*data_set[dim][index]
        q.append(q_value)
        
    for i in range(k):
        G_row = [0.0]*k
        G_row[i] = -1.0
        G.append(G_row)
        
    h = [0.0]*k
    A = [1.0]*k 
    b = [1.0]
    
    P = matrix(P)
    q = matrix(q)
    G = matrix(G)
    h = matrix(h)
    A = matrix(A,(1,k))#原型为cvxopt.matrix(array,dims)，等价于A = matrix([[1.0],[1.0]]）
    b = matrix([1.0])
    
    W = solvers.qp(P,q,G,h,A,b)['x']
    for i in range(k):
        class_[label[k_index_list[i]]] += W[i]
    return class_.index(max(class_))

In [7]:
train_set,test_set = split_train_test(data_num)

In [ ]:
train_set,test_set = split_train_test(data_num)
k = 1
real_class_list = []
predict_class_list = []
test_num = len(test_set)
hit_num = 0
for index1 in test_set:
    real_class = label[index1]
    real_class_list.append(real_class)
    x = []
    for i in range(dimensions):
        x.append(data_set[i][index1])
    distance_index_list = []
    for index2 in train_set:
        y = []
        for i in range(dimensions):
            y.append(data_set[i][index2])
        distance = Odistance(x,y)
        distance_index_list.append((index2,distance))
    distance_index_list.sort(key = lambda x : x[1]) #从小到大
    k_list = distance_index_list[0:k]
    
    predict_class = judge(k_list)
    predict_class_list.append(predict_class)
    if predict_class == real_class:
        hit_num += 1
        
print("准确率为：",hit_num)
print(real_class_list)
print(predict_class_list)

D:\compiler\anaconda\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in ubyte_scalars
  """


In [ ]:
print("准确率为：",hit_num)
print(real_class_list)
print(predict_class_list)

In [208]:
list_ = [69,60,70,69,75,64,73,74,60,72]
print(sum(list_)/750)
import numpy as np 
var = np.var(list_)
print((var**0.5)/75)

0.9146666666666666
0.06958927120373273
